In [ ]:

import functools
from typing import Callable
from better_partial import _, partial


def compose(f, g):
    return lambda *args, **kw: f(g(*args, **kw))

class PointFreeFunction:
    def __init__(self, f):
        self.f = f
        functools.update_wrapper(self, f)
    
    def f(self, *args, **kw):
        return partial(self.f)(*args, **kw)

    def __call__(self, *args, **kw):
        return self.f(*args, **kw)
    
    def __rshift__(self, other: PointFreeFunction | Callable):
        if isinstance(other, PointFreeFunction):
            return PointFreeFunction(compose(self.f, other.f))
        elif callable(other):
            return PointFreeFunction(compose(self.f, other))
        else:
            raise TypeError("other must be callable or PointFreeFunction")


from jax import vmap
# F = lambda f: PointFreeFunction(partial(f))
F  = PointFreeFunction
vmap = compose(F, vmap)